# Wind Load Design for Flat Roofs

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import math
import numpy as np
import plotly as pl
import plotly.graph_objects as go
import seaborn
import pandas

import matplotlib.pyplot as plt

import fpdf

In [5]:
# Layout ("P", "L")
# Unit ("mm", "cm", "in")
# format ("A3", "A4" (default), "A5", "Letter", "Legal", (100, 150))

title = "Dunfermline Solar Farm Wind Loading Report"


class PDF(fpdf.FPDF):
    def __init__(self, orientation='P', unit='mm', format='A4'):
        super().__init__(orientation, unit, format)
        self.set_left_margin(25)
        self.set_right_margin(25)
        
        
    def header(self):
        if self.page == 1:
            self.set_font("helvetica", "", 18)
            title = "Dunfermline Solar Farm Wind Loading Report"
            self.title_w = self.get_string_width(title) + 6
            self.cell(self.title_w, 10, title, border=False, align="L", fill=0)
            self.ln(10)
            self.set_font("helvetica", "", 12)
            self.cell(self.title_w, 10, "MEng/CEng Mihail Todorov", border=False, align="L", fill=0)
            self.ln(5)
            self.cell(self.title_w, 10, "Belectric UK", border=False, align="L", fill=0)
            self.ln(15)
            self.image("solar_image.jpg", x=(self.w - 210) / 2, y=self.get_y(), w=210)
            self.image("wind_image.jpg", x=(self.w - 210) / 2, y=self.get_y() + 80, w=210)
            self.ln(130)
    
    def footer(self):
        self.set_y(-15)
        self.set_font("helvetica", "", 10)
        self.set_text_color(0, 0, 0)
        self.cell(0, 10, f"Page {self.page_no()}/{{nb}}", align="C")

    def chapter_title(self, name):
        pass
        
    def chapter_body(self, name):
        
        with open(name, "rb") as fh:
            txt = fh.read().decode("latin-1")
            
        self.set_font("helvetica", "", 12)
        
        self.multi_cell(0, 5, txt)
        
        self.ln(10)

        self.cell(15, 10, "END OF CHAPTER")
        self.ln(20)
        
    def add_section_from_file(self, title, file_path):
        self.add_page()
        self.ln(20)
        self.section_title = title
        self.set_font("helvetica", "B", 12)
        self.cell(0, 10, title, border=False, ln=True)
        self.ln(10)
        self.set_font("helvetica", "", 12)
        
        with open(file_path, "r") as file:
            content = file.read()
        self.multi_cell(0, 10, content)
        self.ln(5)
        
    def add_mixed_section(self, title, file_path, content_dict):
        self.add_page()
        toc.append((title, self.page_no()))
        self.ln(10)
        self.set_font("helvetica", "B", 12)
        self.cell(0, 10, title, border=False, ln=True)
        self.ln(15)
        self.set_font("helvetica", "", 12)
        
        # Add text from file
        with open(file_path, "r") as file:
            content = file.read()
        self.multi_cell(0, 10, content)
        self.ln(5)
        
        # Add dictionary values
        for label, value in content_dict.items():
            self.cell(100, 10, label, border=False)
            self.cell(0, 10, f"{value}", border=False, ln=True, align='R')

        self.ln(5)
        

def get_user_input():
    H = float(input("Enter the Building Height (H) in meters: "))
    L = float(input("Enter the Building Length (L) in meters: "))
    W = float(input("Enter the Building Width (W) in meters: "))
    h = float(input("Enter the Height above Ground (h) in meters: "))
    L_distance_shore = float(input("Enter the Distance from Building to Shoreline (L_distance_shore) in meters: "))
    A_altitude = float(input("Enter the Altitude above Sea Level (A_altitude) in meters: "))
    V_b_map_wind_velocity = float(input("Enter the Basic Wind Velocity (V_b_map_wind_velocity) in m/s: "))
    c_dir_factor = float(input("Enter the Directional Factor (c_dir_factor): "))
    c_season_factor = float(input("Enter the Season Factor (c_season_factor): "))
    K_shape_parameter = float(input("Enter the Shape Parameter (K_shape_parameter): "))
    n_exponent = float(input("Enter the Exponent (n_exponent): "))
    p_air_density = float(input("Enter the Air Density (p_air_density) in kg/m^3: "))
    p_probability_of_annual_exceedance = float(input("Enter the Probability of Annual Exceedance (p_probability_of_annual_exceedance): "))

    
class WindLoadDesignFlatRoof:
    def __init__(self, project_name, H, L, W, h, L_distance_shore, A_altitude, V_b_map_wind_velocity, c_dir_factor, c_season_factor, K_shape_parameter, n_exponent, p_air_density, p_probability_of_annual_exceedance):
        self.project_name = project_name
        
        self.H = H  # Height of the building
        self.L = L  # Length of the building
        self.W = W  # Width of the building
        self.h = h  # Height above the ground level at which the wind speed is required
        self.L_distance_shore = L_distance_shore  # Distance from the building location to the shoreline
        self.A_altitude = A_altitude  # Altitude above sea level at the building location
        self.V_b_map_wind_velocity = V_b_map_wind_velocity  # Basic wind velocity at the building location
        self.c_dir_factor = c_dir_factor  # Directional factor
        self.c_season_factor = c_season_factor  # Season factor
        self.K_shape_parameter = K_shape_parameter  # Shape parameter for the building
        self.n_exponent = n_exponent  # Exponent for the building shape
        self.p_air_density = p_air_density  # Air density
        self.p_probability_of_annual_exceedance = p_probability_of_annual_exceedance  # Probability of annual exceedance
        
        self.calculate_additional_values()

    def calculate_additional_values(self):
        self.A_ref_area = self.L * self.W * math.sin(math.radians(55))
        self.A_uplift_area = self.L * self.W * math.cos(math.radians(55))
        
        self.c_alt_factor = self.A_altitude * 0.001 + 1
        self.v_b_0_fundamental_wind_velocity = self.c_alt_factor * self.V_b_map_wind_velocity
        self.c_prob_factor = ((1 - self.K_shape_parameter * np.log(-np.log(1 - self.p_probability_of_annual_exceedance))) /
                              (1 - self.K_shape_parameter * (np.log(-np.log(0.98))))) ** self.n_exponent
        self.v_b_basic_wind_velocity = (self.c_dir_factor * self.c_season_factor *
                                         self.v_b_0_fundamental_wind_velocity * self.c_prob_factor)
        self.q_b_reference_mean_velocity_pressure = 0.5 * self.p_air_density * (self.v_b_basic_wind_velocity ** 2)
        self.c_e_exposure_factor = 1.5
                                                                                
        self.q_p_peak_velocity_pressure = self.q_b_reference_mean_velocity_pressure * self.c_e_exposure_factor
        self.F_wind_total_wind_force = self.c_season_factor * self.c_dir_factor * self.q_p_peak_velocity_pressure
        
         # Calculate additional values for bending moment, uplift, and shear
        self.V_total_shear_force = self.F_wind_total_wind_force * self.A_ref_area
        self.M_total_bending_moment = self.h/2 * self.V_total_shear_force
        self.U_total_uplift = self.F_wind_total_wind_force * self.A_uplift_area


    def print_results(self):
        
        print("WIND LOAD DESIGN RESULTS")
        print(f"------------------------------")
        print()
        
        # Geometry
        print(f"------------------------------")
        print("Geometry")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Structure Height (H):", f"{self.H} m"))
        print("{:<80} {:>10}".format("Structure Length (L):", f"{self.L} m"))
        print("{:<80} {:>10}".format("Structure Width (W):", f"{self.W} m"))
        print("{:<80} {:>10}".format("Height above Ground (h):", f"{self.h} m"))
        
        print()
        # Site Specific Factors
        print(f"------------------------------")
        print("Site Specific Factors")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Distance from Building to Shoreline (L_distance_shore):", f"{self.L_distance_shore} m"))
        print("{:<80} {:>10}".format("Altitude above Sea Level (A_altitude):", f"{self.A_altitude} m"))
        print("{:<80} {:>10}".format("Basic Wind Velocity (V_b_map_wind_velocity):", f"{self.V_b_map_wind_velocity} m/s"))
        print("{:<80} {:>10}".format("Directional Factor (c_dir_factor):", self.c_dir_factor))
        print("{:<80} {:>10}".format("Season Factor (c_season_factor):", self.c_season_factor))
        print("{:<80} {:>10}".format("Shape Parameter (K_shape_parameter):", self.K_shape_parameter))
        print("{:<80} {:>10}".format("Exponent (n_exponent):", self.n_exponent))
        print("{:<80} {:>10}".format("Air Density (p_air_density):", f"{self.p_air_density} kg/m^3"))
        print("{:<80} {:>10}".format("Probability of Annual Exceedance (p_probability_of_annual_exceedance):", self.p_probability_of_annual_exceedance))
        print("{:<80} {:>10}".format("Altitude Factor (c_alt_factor):", f"{self.c_alt_factor}"))
        
        print()
        # Velocity and Pressure Profile
        print(f"------------------------------")
        print("Velocity and Pressure Profile")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Fundamental Wind Velocity (V_b_0):", f"{self.v_b_0_fundamental_wind_velocity:.2f} m/s"))
        print("{:<80} {:>10}".format("Probability Factor (c_prob_factor):", f"{self.c_prob_factor:.2f}"))
        print("{:<80} {:>10}".format("Basic Wind Velocity (V_b):", f"{self.v_b_basic_wind_velocity:.2f} m/s"))
        print("{:<80} {:>10}".format("Reference Mean Velocity Pressure (q_b):", f"{self.q_b_reference_mean_velocity_pressure:.2f} N/m^2"))
        print("{:<80} {:>10}".format("Peak Velocity Pressure (q_p):", f"{self.q_p_peak_velocity_pressure:.2f} N/m^2"))
        print("{:<80} {:>10}".format("Total Wind Force per unit area (F_total):", f"{self.F_wind_total_wind_force:.2f} N/m^2"))
        
        print()
        # Velocity and Pressure Profile
        print(f"------------------------------")
        print("Static Loading")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Total Shear Force on the Structure:", f"{self.V_total_shear_force:.2f} N"))
        print("{:<80} {:>10}".format("Total Bending Moment on the Structure:", f"{self.M_total_bending_moment:.2f} Nm"))
        print("{:<80} {:>10}".format("Total Uplift on the Structure:", f"{self.U_total_uplift:.2f} N"))

    

    def generate_wind_loading_report(self):
        self.pdf = PDF("P", "mm", format="A4")
        self.pdf.alias_nb_pages()
        self.pdf.set_auto_page_break(auto=1, margin=15)
        toc = []

        def add_section(title, content_dict):
            self.pdf.add_page()
            toc.append((title, self.pdf.page_no()))
            self.pdf.ln(10)
            self.pdf.set_font("helvetica", "B", 12)
            self.pdf.cell(0, 10, title)
            self.pdf.ln(15)
            self.pdf.set_font("helvetica", "", 12)

            for label, value in content_dict.items():
                self.pdf.cell(100, 10, label, border=False)
                self.pdf.cell(0, 10, f"{value}", border=False, ln=True, align='R')

            self.pdf.ln(5)

        def add_table_of_contents():
            self.pdf.set_font("helvetica", "B", 16)
            self.pdf.cell(0, 10, "Table of Contents", align='C')
            self.pdf.ln(10)
            self.pdf.set_font("helvetica", "", 12)
            for i, (title, page) in enumerate(toc, start=1):
                self.pdf.cell(0, 10, f"{i}. {title}", border=False)
                self.pdf.set_x(-30)
                self.pdf.cell(0, 10, f"{page}", border=False)
                self.pdf.ln(5)

        # Add the first page with header and images
        self.pdf.add_page()
        self.pdf.set_font("helvetica", size=12)
        self.pdf.ln(130)  # Adjust this value to ensure text starts after the images

        # Add a blank TOC page
        self.pdf.add_page()
        toc_page = self.pdf.page_no()

        # Collect content
        geometry = {
            "Structure Height (H):": f"{flat_roof.H} m",
            "Structure Length (L):": f"{flat_roof.L} m",
            "Structure Width (W):": f"{flat_roof.W} m",
            "Height above Ground (h):": f"{flat_roof.h} m"
        }

        site_factors = {
            "Distance from Building to Shoreline (L_distance_shore):": f"{flat_roof.L_distance_shore} m",
            "Altitude above Sea Level (A_altitude):": f"{flat_roof.A_altitude} m",
            "Basic Wind Velocity (V_b_map_wind_velocity):": f"{flat_roof.V_b_map_wind_velocity} m/s",
            "Directional Factor (c_dir_factor):": flat_roof.c_dir_factor,
            "Season Factor (c_season_factor):": flat_roof.c_season_factor,
            "Shape Parameter (K_shape_parameter):": flat_roof.K_shape_parameter,
            "Exponent (n_exponent):": flat_roof.n_exponent,
            "Air Density (p_air_density):": f"{flat_roof.p_air_density} kg/m^3",
            "Probability of Annual Exceedance (p_probability_of_annual_exceedance):": flat_roof.p_probability_of_annual_exceedance,
            "Altitude Factor (c_alt_factor):": f"{flat_roof.c_alt_factor}"
        }

        velocity_pressure_profile = {
            "Fundamental Wind Velocity (V_b_0):": f"{flat_roof.v_b_0_fundamental_wind_velocity:.2f} m/s",
            "Probability Factor (c_prob_factor):": f"{flat_roof.c_prob_factor:.2f}",
            "Basic Wind Velocity (V_b):": f"{flat_roof.v_b_basic_wind_velocity:.2f} m/s",
            "Reference Mean Velocity Pressure (q_b):": f"{flat_roof.q_b_reference_mean_velocity_pressure:.2f} N/m^2",
            "Peak Velocity Pressure (q_p):": f"{flat_roof.q_p_peak_velocity_pressure:.2f} N/m^2",
            "Total Wind Force per unit area (F_total):": f"{flat_roof.F_wind_total_wind_force:.2f} N/m^2"
        }

        static_loading = {
            "Total Shear Force on the Structure:": f"{flat_roof.V_total_shear_force:.2f} N",
            "Total Bending Moment on the Structure:": f"{flat_roof.M_total_bending_moment:.2f} Nm",
            "Total Uplift on the Structure:": f"{flat_roof.U_total_uplift:.2f} N"
        }
        
        self.pdf.add_section_from_file("Scope", "scope.txt")
        add_section("Geometry", geometry)
        add_section("Site Specific Factors", site_factors)
        add_section("Velocity and Pressure Profile", velocity_pressure_profile)
        add_section("Static Loading", static_loading)
        self.pdf.add_section_from_file("Conclusions", "scope_2.txt")

        # Save the current page number
        current_page = self.pdf.page_no()

        # Go back to the TOC page and fill it
        self.pdf.page = toc_page
        self.pdf.set_auto_page_break(auto=0)
        add_table_of_contents()
        self.pdf.set_auto_page_break(auto=1, margin=15)

        # Restore to the last page after TOC
        self.pdf.page = current_page

        self.pdf.output(f"{self.project_name}_Wind_Loading_Report_R01.pdf")
        
        
# User Input usage:
# get_user_input()

# Example usage:
flat_roof = WindLoadDesignFlatRoof(
    project_name = "Dunfermline",
    H=4, L=3.313, W=4, h=5, L_distance_shore=30, A_altitude=115, V_b_map_wind_velocity=24.5,
    c_dir_factor=1.0, c_season_factor=1.0, K_shape_parameter=0.2, n_exponent=0.5, p_air_density=1.25,
    p_probability_of_annual_exceedance=0.02
)


flat_roof.calculate_additional_values()
flat_roof.print_results()
flat_roof.generate_wind_loading_report()

WIND LOAD DESIGN RESULTS
------------------------------

------------------------------
Geometry
------------------------------
Structure Height (H):                                                                   4 m
Structure Length (L):                                                               3.313 m
Structure Width (W):                                                                    4 m
Height above Ground (h):                                                                5 m

------------------------------
Site Specific Factors
------------------------------
Distance from Building to Shoreline (L_distance_shore):                                30 m
Altitude above Sea Level (A_altitude):                                                115 m
Basic Wind Velocity (V_b_map_wind_velocity):                                       24.5 m/s
Directional Factor (c_dir_factor):                                                      1.0
Season Factor (c_season_factor):                   

PermissionError: [Errno 13] Permission denied: 'Dunfermline_Wind_Loading_Report_R01.pdf'